In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
fp = '/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds'
# fp = '/Users/zachkeskinen/Documents/spicy-snow/data/10_img_dB.pkl'
with open(fp, 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.snow_index import calc_delta_VV, calc_delta_cross_ratio

# ds = ds.isel(time = slice(1, 3))

In [ ]:
def calc_delta_cross_ratio(dataset: xr.Dataset, A: float = 2, inplace: bool = False) -> xr.Dataset:
    """
    Calculate change in cross-polarization ratio for all time steps.
    
    delta-gamma-cr (i, t) = gamma-cr (i, t) - gamma-cr (i, t_previous)

    with:
    gamma-cr = A * VH - VV

    and gamma-VH and gamma-VV in dB. Lieven's et al. 2021 tests A over [1, 2, 3]
    and fit to A = 2.

    Returns nans in time slice of deltaCR with no previous image 
    (first of each relative orbit)

    Args:
    dataset: Xarray Dataset of sentinel images
    A: fitting parameter [default = 2]
    inplace: operate on dataset in place or return copy

    Returns:
    dataset: Xarray Dataset of sentinel images with deltaCR added as data var
    """

    # check inplace flag
    if not inplace:
        dataset = dataset.copy(deep=True)

    # check for amp
    if 's1_units' in dataset.attrs.keys():
        assert dataset.attrs['s1_units'] == 'dB', 'Sentinel-1 units must be in dB'

    # calculate cross ratio of VH to VV with fitting parameter A
    gamma_cr = (A * dataset['s1'].sel(band='VH')) - dataset['s1'].sel(band='VV')

    return gamma_cr

    # get all unique relative orbits
    orbits = np.unique(dataset['relative_orbit'].values)
    
    # Identify previous image from the same relative orbit (6, 12, 18, or 24 days ago)
    for orbit in orbits:

        # Calculate change in gamma-cr between previous and current time step
        diffCR = gamma_cr.sel(time = dataset.relative_orbit == orbit).diff(dim = 'time')
        
        # add delta-gamma-cr as band to dataset
        # if adding new 
        if 'deltaCR' not in dataset.data_vars:
            # add delta-gamma-CR as new variable to dataset
            dataset['deltaCR'] = diffCR
        
        else:
            # update deltaCR times with this. 
            dataset['deltaCR'].loc[dict(time = diffCR.time)] = diffCR
    
    if not inplace:
        return dataset

In [ ]:
ds['s1'].isel(time = 0).sel(band = 'VH')*2 -ds['s1'].isel(time = 0).sel(band = 'VV') 

In [ ]:
ds1 = calc_delta_cross_ratio(ds)
# ds1 = calc_delta_VV(ds1)

In [ ]:
ds1.isel(time = 0)

In [ ]:
CR1

In [ ]:
test_A = 2

CR1, CR2, CR3 = ds['s1'].sel(band = 'VV') * test_A - ds['s1'].sel(band = 'VH')
real2_1_diff = CR2 - CR1
real3_2_diff = CR3 - CR2

In [ ]:
ds1['deltaCR'].isel(time = 1)

In [ ]:
assert np.allclose(ds1['deltaCR'].isel(time = 1), real2_1_diff)